## Loading data to TFRecords
https://medium.com/coinmonks/beginners-guide-to-feeding-data-in-tensorflow-part2-5e2506d75429
https://medium.com/@cl2227619761/creating-tensorflow-tfrecords-file-using-images-ade8f3076131

The various advantages of TFRecords format are —
* Faster to read than PNG or other formats.
* Gives you the flexibility in data pipelining.
* Useful when the data doesn’t fit in memory.
* Less no of files to manage.(Personal opinion)

In [30]:
import tensorflow as tf
import os
from PIL import Image  # used to read images from directory
import pandas as pd
import numpy as np

In [32]:
test_path = 'test_images'
train_path = 'train_images'
IMG_WIDTH = 1600
IMG_HEIGHT = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
# my file path, just like the picture above
image_path = "train_images/"
# the tfrecord file path, you need to create the folder yourself
recordPath = "tfrecord/"
# the best number of images stored in each tfrecord file
bestNum = 1000
# the index of images flowing into each tfrecord file
num = 0
# the index of the tfrecord file
recordFileNum = 0
# name format of the tfrecord files
recordFileName = ("train.tfrecords-%.3d" % recordFileNum)
# tfrecord file writer
writer = tf.io.TFRecordWriter(recordPath + recordFileName)

In [18]:
defects_df = pd.read_csv('train.csv')
defects_df['ImageId'] = defects_df['ImageId_ClassId'].apply(lambda x : x.split('_')[0])
defects_df['ClassId'] = defects_df['ImageId_ClassId'].apply(lambda x : x.split('_')[1])
defects_df.set_index('ImageId_ClassId', inplace=True)

In [19]:
# 1 - red, 2 - yellow, 3- green , 4 - blue
colors = {1 : [255, 0, 0], 2: [255, 255, 0], 3: [0, 255, 0], 4: [0, 0, 255]}

In [20]:
## Unique images with defects
no_defects = defects_df[defects_df['EncodedPixels'].notnull()]
len(no_defects['ImageId'].unique())

6666

In [21]:
'''
This function is an rle to mask converter.
You supply the rle string for one mask, along with it's height and width
from the solutions for stage1 and it will return the mask.
'''
def rle2mask(rle, imgshape):
    width = imgshape[0]
    height= imgshape[1]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return np.flipud( np.rot90( mask.reshape(height,width), k=1 ) )

In [36]:
def img_with_mask(img):
    #print(image)
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH))
    #find defects for given image
    df = no_defects[no_defects['ImageId'] == img].reset_index()
    for j in range(len(df)):
        mask = rle2mask( df['EncodedPixels'].iloc[j], (img.shape[0], img.shape[1])  )
        classes[mask==1] = colors[int(df['ClassId'].iloc[j])]
    #masks.append(classes)
    return mask

In [ ]:
print("Creating the 000 tfrecord file")
for img_name in os.listdir(train_path):
    num += 1
    if num > bestNum:
        num = 1
        recordFileNum += 1
        writer = tf.io.TFRecordWriter(recordPath + str(recordFileNum))
        print("Creating the %.3d tfrecord file" % recordFileNum)
    img = Image.open(train_path + '/' + img_name, "r")
    img_raw = img.tobytes()
    example = tf.train.Example(features=tf.train.Features(feature={
        "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        "img_mask": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_with_mask(img_raw).tobytes()]))}))
    writer.write(example.SerializeToString())
writer.close()

Creating the 000 tfrecord file
Creating the 002 tfrecord file


In [ ]:
path = os.path.abspath('train_masks/')
def _ondisk_parse_(filename):
    filename = tf.cast([filename],tf.string)
    
    label = tf.string_split([tf.string_split(filename,'_').values[1]],'.').values[0]
    label = tf.string_to_number([label],tf.int32)
    
    path = os.path.abspath('./digit-recognizer/train//')
    path = tf.cast([path],tf.string)
    
    final_path = tf.string_join((path,tf.cast(['/'],tf.string),filename))
    
    image_string = tf.read_file(final_path[0])
    image = tf.image.decode_jpeg(image_string)
    image = tf.cast(image,tf.int8)
    image = tf.cast(image,tf.float32)
    image_reshaped = tf.reshape(image,(784,))
    return (dict({'image':image}),label)